### Demo: converting OPHYS to NWB and then extracting relevant metadata for analysis
Neurodata Without Borders (NWB) is a powerful tool for organizing your data for purposes such as:
* data sharing
* result reproduction
* experiment organization

Here is a demo showing its utility:
_____

### Relevant packages
For Ophys, we must have the following packages:
* 'pynwb'
* 'scikit-image'
* matplotlib


In [1]:
# This code will generate an NWB file for ophys data
from datetime import datetime
from uuid import uuid4
import numpy as np
from dateutil import tz
from pynwb import NWBHDF5IO, NWBFile, TimeSeries
from pynwb.behavior import Position, SpatialSeries
from pynwb.epoch import TimeIntervals
from pynwb.file import Subject

#import cv2
from skimage import io

import matplotlib.pyplot as plt
import numpy as np
from dateutil.tz import tzlocal

from pynwb import NWBHDF5IO, NWBFile, TimeSeries
from pynwb.image import ImageSeries
from pynwb.ophys import (
    CorrectedImageStack,
    Fluorescence,
    ImageSegmentation,
    MotionCorrection,
    OnePhotonSeries,
    OpticalChannel,
    RoiResponseSeries,
    TwoPhotonSeries,
)

### Lets build our NWB file using the ``NWBFile`` object

In [2]:
session_start_time = datetime(2018, 4, 25, 2, 30, 3, tzinfo=tz.gettz("US/Pacific"))
# initialize the nwbfile
nwbfile = NWBFile(
    session_description=input("Enter description of your recording session: "),  # required
    identifier=str(uuid4()),  # required
    session_start_time=session_start_time,  # required
    session_id=input("Enter unique identifier for session: "),  # optional
    experimenter=[
        input("Enter experimenter name: "),
    ],  # optional
    lab=input("Enter lab name: "),  # optional
    institution=input("Enter institution name: "),  # optional
    experiment_description=input("Enter a description of your experiment"),  # optional
    related_publications=input("Enter any information about publication (if relevant)"),  # optional
)

### Enter information about subject

In [3]:
# enter information about subject
nwbfile.subject = Subject(
    subject_id=input("Enter subject ID: "),
    age=input("Enter subject age as such (PD100):  "),
    description=input("Enter subject identifier: "),
    species=input("Enter species name: "),
    sex=input("Enter sex of subject: "),
)

### Add our data to the NWB file

Interface with the user to get some information about directory

In [4]:
# directory information
folder_name = input("Enter the folder name for your data: ")
fname_neuron = input("Enter file name with extension: ")
frame_rate = float(input("Enter the frame rate: "))

Using the scikit-image, we can load in our data with io


In [5]:
# read data
data = io.imread(folder_name+'/'+fname_neuron)

Lets view the video for fun

### Now lets add the video to the NWB file

In [6]:
# create device
device = nwbfile.create_device(
    name="Microscope",
    description="My two-photon microscope",
    manufacturer="The best microscope manufacturer",
)
optical_channel = OpticalChannel(
    name="OpticalChannel",
    description="an optical channel",
    emission_lambda=525.0,
)

### Create imagingplane

In [7]:
# create imagingplane object
imaging_plane = nwbfile.create_imaging_plane(
    name="ImagingPlane",
    optical_channel=optical_channel,
    imaging_rate=frame_rate,
    description="Activation of cells",
    device=device,
    excitation_lambda=600.0,
    indicator="GFP",
    location="Somewhere",
    grid_spacing=[0.01, 0.01],
    grid_spacing_unit="meters",
    origin_coords=[1.0, 2.0, 3.0],
    origin_coords_unit="meters",
)

### Store our 1p data

In [8]:
one_p_series1 = OnePhotonSeries(
    name="CalciumDye",
    data=data,
    imaging_plane=imaging_plane,
    rate=10.0,
    unit="pixels",
)

### Save output

In [15]:
nwbfile.add_acquisition(one_p_series1)
with NWBHDF5IO(folder_name+"/data_ophys_nwb.nwb", "w") as io:
    io.write(nwbfile)

ValueError: 'CalciumDye' already exists in NWBFile 'root'

In [19]:
np.save(folder_name+"/data_ophys.npy",data)
bad_frames = np.array([20,30,40])
np.save(folder_name+'/bad_frames.npy', bad_frames)

### Lets visualize the results

In [14]:
nwbfile

root pynwb.file.NWBFile at 0x6056640416
Fields:
  acquisition: {
    CalciumDye <class 'pynwb.ophys.OnePhotonSeries'>
  }
  devices: {
    Microscope <class 'pynwb.device.Device'>
  }
  experimenter: ['']
  file_create_date: [datetime.datetime(2023, 8, 18, 14, 9, 37, 503580, tzinfo=tzlocal())]
  identifier: 96bc0263-93f9-4355-91e1-595fdd61035c
  imaging_planes: {
    ImagingPlane <class 'pynwb.ophys.ImagingPlane'>
  }
  related_publications: ['']
  session_description: kjl;'
  session_start_time: 2018-04-25 02:30:03-07:00
  subject: subject pynwb.file.Subject at 0x4389383376
Fields:
  age__reference: birth

  timestamps_reference_time: 2018-04-25 02:30:03-07:00